# Checking Storage Accounts

This notebook will check that the storage accounts are using the Express Route

## Make an Azure connection
Next we need to make a connection to Azure for the PowerShell session. The cell below is going to do this but it has an idiosyncrasy. When you press the play button it will just spin

![image](.\Images\01-Connect-Azure.png)  

You will need to look for the sign in pop-up *behind* Azure Data Studio  

![image](.\Images\02-Hidden-Window.png)

Enter your credentials and then you will see a similar output to this if all is successful

![image](.\Images\04-SuccessfulAzureConnection.png)



In [1]:
# Connect to Azure with a browser sign in token
Connect-AzAccount


Account                  SubscriptionName         TenantId                     
-------                  ----------------         --------                     
rob.sewell@uk.zurich.com GROUP - EDAA-EuDL - PROD 473672ba-cd07-4371-a2ae-78...




# Check and set the subscription

Next, we need to list the subscriptions that you ahve access to and decide which one to use. This code will list the subscriptions that you have access to.

In [11]:
Get-AzSubscription


Name                        Id                                   TenantId      
----                        --                                   --------      
GROUP - EDAA-EuDL - PROD    c60c421b-8ec3-40df-8ce7-54daa45ca323 473672ba-cd...
GROUP - EDAA-EuDL - NONPROD 1ce00944-b7bb-4394-8303-8f43dba9e520 473672ba-cd...




Choose the subscription that you want to use from the results above amd copy the ID and use it in the below code

In [12]:
$ID = '1ce00944-b7bb-4394-8303-8f43dba9e520'
Set-AzContext -SubscriptionId $ID


Name                                     Account   Subscript Environme TenantId
                                                   ionName   nt                
----                                     -------   --------- --------- --------
GROUP - EDAA-EuDL - NONPROD (1ce00944... rob.se... GROUP ... AzureC... 47367...




## Listing Resource Groups

We can list the resource groups that we have access to 

In [13]:
Get-AzResourceGroup | Format-Table


ResourceGroupName        Location    ProvisioningState Tags                    
-----------------        --------    ----------------- ----                    
edaa-eudl-nonprod-sql-rg northeurope Succeeded         {creator, project, bi...
eudl-gldlsa-dev-rg       northeurope Succeeded         {creator, project, bi...
eudl-ukgida-dev-rg       northeurope Succeeded         {project, billing, cr...




# List Storage Accounts

We can list the storage accounts we have access to.

Add the resource group names from the above results

In [18]:
$RGNames = 'edaa-eudl-nonprod-sql-rg','eudl-gldlsa-dev-rg','eudl-ukgida-dev-rg'
$StorageAccounts = $RGNames.ForEach{
    Get-AzStorageAccount -ResourceGroupName $psitem 
}
$StorageAccounts | Format-Table


StorageAccountName     ResourceGroupName        PrimaryLocation SkuName      Ki
                                                                             nd
------------------     -----------------        --------------- -------      --
sqlvarh4osr6siy2mk     edaa-eudl-nonprod-sql-rg westeurope      Standard_LRS St
eudlsecuritydevstore   edaa-eudl-nonprod-sql-rg northeurope     Standard_LRS St
infaauditlogstore      edaa-eudl-nonprod-sql-rg northeurope     Standard_LRS St
auditfwstorage         eudl-gldlsa-dev-rg       westeurope      Standard_LRS St
auditfw                eudl-gldlsa-dev-rg       northeurope     Standard_LRS St
eudlukgidaml2634684242 eudl-gldlsa-dev-rg       northeurope     Standard_LRS St
eudlukgidadevstore     eudl-ukgida-dev-rg       northeurope     Standard_LRS St




If there are no results, you do not have access to see the storage accounts or there are no storage accounts

In [22]:
$StorageAccounts | ForEach-Object {
    $AzureAllowed = @{Name = 'AzureAllowed';Expression = {$Psitem.NetworkRuleSet.Where{$_.ByPass -eq 'AzureServices'}.DefaultAction}}
    $VirtualNetworkRules = @{Name = 'VNetRules';Expression = {if($Psitem.NetworkRuleSet.VirtualNetworkRules){'Yes'}else{'No'} }}
    $Psitem | Select ResourceGroupName ,StorageAccountName , $AzureAllowed, $VirtualNetworkRules
}


ResourceGroupName        StorageAccountName     AzureAllowed VNetRules
-----------------        ------------------     ------------ ---------
edaa-eudl-nonprod-sql-rg sqlvarh4osr6siy2mk            Allow No       
edaa-eudl-nonprod-sql-rg eudlsecuritydevstore           Deny Yes      
edaa-eudl-nonprod-sql-rg infaauditlogstore             Allow No       
eudl-gldlsa-dev-rg       auditfwstorage                Allow No       
eudl-gldlsa-dev-rg       auditfw                       Allow No       
eudl-gldlsa-dev-rg       eudlukgidaml2634684242        Allow No       
eudl-ukgida-dev-rg       eudlukgidadevstore            Allow No       




In [10]:
Get-AzStorageAccount